In [1]:
import sys
sys.path.append('../../')
from taxcalc import *
import numpy as np
import pandas as pd
import copy

In [103]:
def make_calculator(refdict = {}, year=2018):
    # Creates a calculator advanced to the given year and calculates tax results
    # Note: Passing an empty dictionary to refdict produces a current law calculator
    assert year in range(2014, 2027)
    assert type(refdict) is dict
    pol = Policy()
    beh = Behavior()
    rec = Records('../../puf.csv')
    if refdict != {}:
        pol.implement_reform(refdict)
    calc1 = Calculator(pol, rec, beh)
    calc1.advance_to_year(year)
    calc1.calc_all()
    return calc1

dropbox_path = 'C:/Users/cody_/Dropbox/TCJA info/notebook & tax calc code/'
# Compile code for distributional analysis
exec(open(dropbox_path + 'distributional_code.py').read())


In [3]:
param = Calculator.read_json_param_objects('../../taxcalc/reforms/2017_law.json', None)
calc_pre = make_calculator(param['policy'], 2018)
calc_tcja = make_calculator({}, 2018)

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.
You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [ ]:
# Distribution of comparable income measure to compare against SCF distribution
calc1 = make_calculator({}, 2015)

expinc = calc1.array('expanded_income')
agi = calc1.array('c00100')
wgt = calc1.array('s006')
compinc = (calc1.array('e00200') + calc1.array('e02100') + calc1.array('e00900') + calc1.array('e02000') + 
           calc1.array('e00400') + calc1.array('e00300') + calc1.array('e00600') + calc1.array('e02300') + 
           calc1.array('e01500') + calc1.array('e02400'))
(expinc_a, wgt_a) = zip(*sorted(zip(expinc, wgt)))
(agi_b, wgt_b) = zip(*sorted(zip(agi, wgt)))
(compinc_c, wgt_c) = zip(*sorted(zip(compinc, wgt)))
expinc_a = np.array(expinc_a)
agi_b = np.array(agi_b)
compinc_c = np.array(compinc_c)
wgt_a = np.array(wgt_a)
wgt_b = np.array(wgt_b)
wgt_c = np.array(wgt_c)
wgt_acum = np.cumsum(wgt_a) / sum(wgt_a)
wgt_bcum = np.cumsum(wgt_b) / sum(wgt_b)
wgt_ccum = np.cumsum(wgt_c) / sum(wgt_c)

qlist_exp = np.ones(9)
qlist_agi = np.ones(9)
qlist_comp = np.ones(9)
qlist_exp[0] = max(expinc_a[wgt_acum <= 0.01])
qlist_agi[0] = max(agi_b[wgt_bcum <= 0.01])
qlist_comp[0] = max(compinc_c[wgt_ccum <= 0.01])
qlist_exp[1] = max(expinc_a[wgt_acum <= 0.05])
qlist_agi[1] = max(agi_b[wgt_bcum <= 0.05])
qlist_comp[1] = max(compinc_c[wgt_ccum <= 0.05])
qlist_exp[2] = max(expinc_a[wgt_acum <= 0.1])
qlist_agi[2] = max(agi_b[wgt_bcum <= 0.1])
qlist_comp[2] = max(compinc_c[wgt_ccum <= 0.1])
qlist_exp[3] = max(expinc_a[wgt_acum <= 0.25])
qlist_agi[3] = max(agi_b[wgt_bcum <= 0.25])
qlist_comp[3] = max(compinc_c[wgt_ccum <= 0.25])
qlist_exp[4] = max(expinc_a[wgt_acum <= 0.5])
qlist_agi[4] = max(agi_b[wgt_bcum <= 0.5])
qlist_comp[4] = max(compinc_c[wgt_ccum <= 0.5])
qlist_exp[5] = max(expinc_a[wgt_acum <= 0.75])
qlist_agi[5] = max(agi_b[wgt_bcum <= 0.75])
qlist_comp[5] = max(compinc_c[wgt_ccum <= 0.75])
qlist_exp[6] = max(expinc_a[wgt_acum <= 0.9])
qlist_agi[6] = max(agi_b[wgt_bcum <= 0.9])
qlist_comp[6] = max(compinc_c[wgt_ccum <= 0.9])
qlist_exp[7] = max(expinc_a[wgt_acum <= 0.95])
qlist_agi[7] = max(agi_b[wgt_bcum <= 0.95])
qlist_comp[7] = max(compinc_c[wgt_ccum <= 0.95])
qlist_exp[8] = max(expinc_a[wgt_acum <= 0.99])
qlist_agi[8] = max(agi_b[wgt_bcum <= 0.99])
qlist_comp[8] = max(compinc_c[wgt_ccum <= 0.99])
for i in range(9):
    print(qlist_exp[i])
for i in range(9):
    print(qlist_agi[i])
for i in range(9):
    print(qlist_comp[i])

## Distributional table for nonprofits

In [97]:
ranking = {'w_adult': 1, 'w_child': 1, 'elast_size': 0.5}
scaling = {'w_adult': 1, 'w_child': 1, 'elast_size': 0}

services_tot = distTable_km(calc_pre, calc_tcja, 'expanded', 'benefits', 'total1', ranking, scaling, ["neginc"], ["",""])
char_tot = distTable_km(calc_pre, calc_tcja, 'expanded', 'charity', 'total1', ranking, scaling, ["neginc"], ["",""])
char_totafter = distTable_km(calc_pre, calc_tcja, 'expanded', 'charity_after', 'total1', ranking, scaling, ["neginc"], ["",""])
wage_tot = distTable_km(calc_pre, calc_tcja, 'expanded', 'wages', 'total1', ranking, scaling, ["neginc"], ["",""])
wage_totafter = distTable_km(calc_pre, calc_tcja, 'expanded', 'wages_after', 'total1', ranking, scaling, ["neginc"], ["",""])
rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']
nonprof_tableA = pd.DataFrame({"Income groups": rowlabel,
                               "Reduced services": services_tot / services_tot[-1],
                               "Reduced compensation": wage_tot / wage_tot[-1],
                               "More giving": char_tot / char_tot[-1],
                               "Preferred mix": (0.208 * services_tot + 0.78 * wage_tot) / (0.208 * services_tot[-1] + 0.78 * wage_tot[-1])})
nonprof_tableB = pd.DataFrame({"Income groups": rowlabel,
                               "Reduced services": services_tot / services_tot[-1],
                               "Reduced compensation": wage_totafter / wage_tot[-1],
                               "More giving": char_totafter / char_tot[-1],
                               "Preferred mix": (0.208 * services_tot + 0.78 * wage_totafter) / (0.208 * services_tot[-1] + 0.78 * wage_tot[-1])})
nonprof_tableA.to_csv(dropbox_path + 'Business tables/nonprof_tableA.csv', index=False)
nonprof_tableB.to_csv(dropbox_path + 'Business tables/nonprof_tableB.csv', index=False)

## Distribution table for state and local governments

In [108]:
ranking = {'w_adult': 1, 'w_child': 1, 'elast_size': 0.5}
scaling = {'w_adult': 1, 'w_child': 1, 'elast_size': 0}
benefits = distTable_km(calc_pre, calc_tcja, 'expanded', 'benefits', 'total1', ranking, scaling, ["neginc"], ["",""])
wage = distTable_km(calc_pre, calc_tcja, 'expanded', 'wages', 'total1', ranking, scaling, ["neginc"], ["",""])
taxes = distTable_km(calc_pre, calc_tcja, 'expanded', 'state_taxes', 'total1', ranking, scaling, ["neginc"], ["",""])
rowlabel = ['Bottom decile', 'Second decile', 'Third decile', 'Fourth decile', 'Fifth decile', 'Sixth decile',
            'Seventh decile', 'Eighth decile', 'Ninth decile', 'Next 5%', 'Next 4%', 'Top 1%', 'All units']
slgtable = pd.DataFrame({"Income groups": rowlabel,
                         "Increase Medicaid spending": benefits / benefits[-1],
                         "Increase compensation": wage / wage[-1],
                         "Reduce taxes": taxes / taxes[-1]})
slgtable.to_csv(dropbox_path + 'Business tables/slg_table.csv', index=False)
slgtable

,Income groups,Increase Medicaid spending,Increase compensation,Reduce taxes
0,Bottom decile,0.009740,0.003439,0.030521
1,Second decile,0.036287,0.012505,0.034537
2,Third decile,0.077335,0.022935,0.036850
3,Fourth decile,0.122424,0.033473,0.040445
4,Fifth decile,0.143391,0.045702,0.046505
5,Sixth decile,0.149513,0.062149,0.053996
6,Seventh decile,0.135787,0.087082,0.067974
7,Eighth decile,0.116482,0.126023,0.092420
8,Ninth decile,0.101497,0.186906,0.142823
9,Next 5%,0.046167,0.132520,0.111478


# Business tax distribution

In [109]:
# Compile code for equity imputation
exec(open(dropbox_path + 'equity_imputation_code.py').read())

In [111]:
equity = imputeEquity(calc_pre)
dshare = imputeDirectEquity(calc_pre)
wtshare = imputeTaxableIndirectEquity(calc_pre, dshare)

In [346]:
# Compile code for business tax distribution
exec(open(dropbox_path + 'business_distribution_code.py').read())

In [347]:
# Choose distribution inputs and assumptions of burden split
ranking = {'w_adult': 1, 'w_child': 1, 'elast_size': 0.5}
scaling = {'w_adult': 1, 'w_child': 1, 'elast_size': 0}
excluding = ["neginc"]
screening = ["", ""]
nonprofit_split = {"services": 0.208, "compensation": 0.78, "donors": 0, "foreign": 0.012}
slgov_split = {"benefits": 0.0, "compensation": 1.0, "taxes": 0.0}

In [348]:
# Results for 2018
dtab18 = fullDistComparison(calc_pre, calc_tcja, 2018, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab18.to_csv(dropbox_path + "Business tables/mainfullstatic2018.csv", index=False)
dtab18

Change in corporate tax liability: -94.38
Change in pre-tax income: 75.64332431812663
Change in after-tax income: 58.759018529820366
Change in corporate tax liability: -94.38
Change in pre-tax income: 75.64332431812663
Change in after-tax income: 242.76805644553144


,Income groups,"IIT, percent","IIT, average","CIT, percent","CIT, average","Both, percent","Both, average"
0,Bottom decile,0.002653,6.987833,0.002830,7.454263,0.005489,14.456917
1,Second decile,0.006629,81.263970,0.001435,17.588834,0.008092,99.208834
2,Third decile,0.014146,262.793798,0.001526,28.338379,0.015726,292.125087
3,Fourth decile,0.015975,402.067450,0.002096,52.743617,0.018105,455.697759
4,Fifth decile,0.013996,465.243522,0.002478,82.373052,0.016521,549.172586
5,Sixth decile,0.014480,630.971772,0.002809,122.391755,0.017354,756.216024
6,Seventh decile,0.015980,893.903075,0.003044,170.283887,0.019116,1069.300692
7,Eighth decile,0.017424,1278.932470,0.003629,266.358251,0.021172,1554.070247
8,Ninth decile,0.017679,1786.378216,0.004295,433.990668,0.022111,2234.268361
9,Next 5%,0.017529,2501.348250,0.005975,852.653050,0.023748,3388.831012


In [349]:
# Results for 2019
dtab19 = fullDistComparison(calc_pre, calc_tcja, 2019, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab19.to_csv(dropbox_path + "Business tables/mainfullstatic2019.csv", index=False)
# Results for 2020
dtab20 = fullDistComparison(calc_pre, calc_tcja, 2020, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab20.to_csv(dropbox_path + "Business tables/mainfullstatic2020.csv", index=False)
# Results for 2021
dtab21 = fullDistComparison(calc_pre, calc_tcja, 2021, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab21.to_csv(dropbox_path + "Business tables/mainfullstatic2021.csv", index=False)
# Results for 2022
dtab22 = fullDistComparison(calc_pre, calc_tcja, 2022, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab22.to_csv(dropbox_path + "Business tables/mainfullstatic2022.csv", index=False)
# Results for 2023
dtab23 = fullDistComparison(calc_pre, calc_tcja, 2023, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab23.to_csv(dropbox_path + "Business tables/mainfullstatic2023.csv", index=False)
# Results for 2024
dtab24 = fullDistComparison(calc_pre, calc_tcja, 2024, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab24.to_csv(dropbox_path + "Business tables/mainfullstatic2024.csv", index=False)
# Results for 2025
dtab25 = fullDistComparison(calc_pre, calc_tcja, 2025, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab25.to_csv(dropbox_path + "Business tables/mainfullstatic2025.csv", index=False)
# Results for 2026
dtab26 = fullDistComparison(calc_pre, calc_tcja, 2026, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab26.to_csv(dropbox_path + "Business tables/mainfullstatic2026.csv", index=False)
# Results for 2027
dtab27 = fullDistComparison(calc_pre, calc_tcja, 2027, equity, dshare, wtshare,
                           nonprofit_split, slgov_split, ranking, scaling, excluding, screening)
dtab27.to_csv(dropbox_path + "Business tables/mainfullstatic2027.csv", index=False)

Change in corporate tax liability: -95.83
Change in pre-tax income: 76.80546481675404
Change in after-tax income: 59.637173852717844
Change in corporate tax liability: -95.83
Change in pre-tax income: 76.80546481675404
Change in after-tax income: 243.86010642455037
Change in corporate tax liability: -79.939
Change in pre-tax income: 64.06920642790784
Change in after-tax income: 49.74804732124555
Change in corporate tax liability: -79.939
Change in pre-tax income: 64.06920642790784
Change in after-tax income: 235.2091900930122
Change in corporate tax liability: -56.961
Change in pre-tax income: 45.65288616746037
Change in after-tax income: 35.45181511484452
Change in corporate tax liability: -56.961
Change in pre-tax income: 45.65288616746037
Change in after-tax income: 222.43523660172332
Change in corporate tax liability: -31.9
Change in pre-tax income: 25.567090970000727
Change in after-tax income: 19.853728659676595
Change in corporate tax liability: -31.9
Change in pre-tax income: 2